In [3]:
!pip install selenium

     ---------------------------------------- 6.5/6.5 MB 10.4 MB/s eta 0:00:00
  Using cached trio-0.22.0-py3-none-any.whl (384 kB)
  Using cached async_generator-1.10-py3-none-any.whl (18 kB)
  Using cached outcome-1.2.0-py2.py3-none-any.whl (9.7 kB)
  Using cached wsproto-1.2.0-py3-none-any.whl (24 kB)
  Using cached h11-0.14.0-py3-none-any.whl (58 kB)


In [9]:
!pip install webdriver-manager

In [70]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import re
import time
from selenium.webdriver.common.keys import Keys
import math
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


path = 'chromedriver'
excutable_path = 'chromedriver.exe'
source_url = "https://map.kakao.com/"
driver = webdriver.Chrome(executable_path=excutable_path)

driver.get(source_url)

search=driver.find_element(By.XPATH, '//*[@id="search.keyword.query"]')
search.send_keys("합정 라멘")
search.send_keys(Keys.ENTER)

time.sleep(2)

html = driver.page_source
soup = BeautifulSoup(html, "html.parser")
moreviews = soup.find_all(name="a", attrs={"class":"moreview"})

driver.close()

urls=[]
for i in moreviews:
    page_url=i.get("href")
    urls.append(page_url)
    
ramen=urls[0]

In [53]:

columns = ['score', 'review']
df = pd.DataFrame(columns=columns)

stars=[]
reviews=[]

driver = webdriver.Chrome(executable_path=excutable_path)
driver.get(ramen)

last_height = driver.execute_script("return document.body.scrollHeight")

while True:
            
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    time.sleep(2)

            # 스크롤 다운 후 스크롤 높이 다시 가져옴
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

    tmp=driver.page_source
    tmp2 = BeautifulSoup(tmp, "html.parser")
    #print(int(tmp2.select('#mArticle > div.cont_evaluation > strong.total_evaluation > span')[0].get_text()))
    total_reviews=int(tmp2.select('#mArticle > div.cont_evaluation > strong.total_evaluation > span')[0].get_text())
    pages=math.ceil(total_reviews/5)
    print(pages)
    time.sleep(5)

for i in range(1,pages+1):
    t1=driver.page_source
    t2=BeautifulSoup(t1, "html.parser")
    t3=t2.find(name="div", attrs={"class":"evaluation_review"})

    star=t3.find_all('em',{'class':'num_rate'})
    review=t3.find_all('p',{'class':'txt_comment'})

    stars.extend(star)
    reviews.extend(review)

    i=i+1
                
    if i > pages:
        break
#     wait = WebDriverWait(driver, 10)
#     element = wait.until(EC.presence_of_element_located((By.XPATH, "//*[@id='mArticle']/div[7]/div[3]/a")))
#     element.send_keys(Keys.ENTER)
    next_page = driver.find_element(By.XPATH, "//*[@id='mArticle']/div[7]/div[3]/a")
    next_page.send_keys(Keys.ENTER)
    time.sleep(1)
    for s, r in zip(stars, reviews):
        row = [s.text[0], r.find(name="span").text]
        series = pd.Series(row, index=df.columns)
        df = df.append(series, ignore_index=True)
    
df.to_csv('oreno_ramen.csv',encoding='utf-8')
driver.close()




5


In [71]:
columns = ['score', 'review']
df = pd.DataFrame(columns=columns)
executable_path = 'C:/chromedriver.exe' # Chrome 드라이버의 실행 파일 경로로 수정해주세요.



stars = []
reviews = []

driver = webdriver.Chrome(executable_path=executable_path)
driver.get(ramen)

last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)

    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

    tmp = driver.page_source
    tmp2 = BeautifulSoup(tmp, "html.parser")
    total_reviews = int(tmp2.select('#mArticle > div.cont_evaluation > strong.total_evaluation > span')[0].get_text())
    pages = math.ceil(total_reviews / 5)
    print(pages)
    time.sleep(5)

    for i in range(1, pages + 1):
        t1 = driver.page_source
        t2 = BeautifulSoup(t1, "html.parser")
        t3 = t2.find(name="div", attrs={"class": "evaluation_review"})

        star = t3.find_all('em', {'class': 'num_rate'})
        review = t3.find_all('p', {'class': 'txt_comment'})

        stars.extend(star)
        reviews.extend(review)

        i = i + 1

        if i > pages:
            break

        next_page = driver.find_element(By.XPATH, "//*[@id='mArticle']/div[7]/div[3]/a")
        next_page.send_keys(Keys.ENTER)
        time.sleep(1)

for s, r in zip(stars, reviews):
    row = [s.text[0], r.find(name="span").text]
    series = pd.Series(row, index=df.columns)
    df = df.append(series, ignore_index=True)

df.to_csv('oreno_ramen.csv', encoding='utf-8')
driver.close()

5
5
5


In [72]:
pd.read_csv('oreno_ramen.csv', encoding='utf-8')

,Unnamed: 0,score,review
